<a href="https://www.kaggle.com/code/stanleyosondu/room-occpancy-prediction?scriptVersionId=106721358" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>